In [21]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

# ETL Pipeline 

### Part I. Pre-Processing the Files

#### Import Python Packages.

In [1]:
# Import Python Packages.
import pandas as pd
import numpy as np
import cassandra
import re
import os
import glob
import json
import csv

#### Creating list of filepath to process all the original csv event files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'
file_path_list = []

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    

C:\Users\Rishabh\Desktop\Preperation2020\Udacity - Data Engineer\Data Modelling with Apache Cassandra


#### Processing the files to create the data file csv that will be used for Apache Cassandra tables.

In [3]:
# initiating an empty list of rows that will have all the data 

full_data_rows_list = []

# for every filepath in the file path list
for f in file_path_list:
    with open(f,'r',encoding='utf8',newline='') as csvfile:
        # creating a csv reader object which is an iterator of lines
        csvreader=csv.reader(csvfile)
        next(csvreader) # reads first line i.e. header so that below code doesn't read the header everytime.
        
        # Extracting each row and append to the full_data_rows_list
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)
            
# print the total number of rows identified above
# print(len(full_data_rows_list)) 
# total 8056 rows.

#### Using above extracted rows to create a full event data csv file called event_datafile_full.csv 
##### It will be used to insert data into the *Apache Cassandra* tables.

In [4]:
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('even_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist', 'firstName', 'gender', 'ItemInSession', 'lastName', 'length',\
                    'level', 'location', 'sessionId', 'song', 'userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow([row[0], row[2], row[3], row[4], row[5], row[6],\
                       row[7], row[8], row[12], row[13], row[16]])

In [5]:
# check the total number of rows added in the csv file
with open('even_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))
# blank rows could not be added.

6821


### Part II. Create the tables in Apache Cassandra to complete the loading and query based analysis of the data

#### event_datafile_new.csv contains all the data required to load into table which contains following columns:

- artist
- firstName of user
- gender
- item numbers in a session
- last name 
- length of the song
- level (paid or free song)
- location of user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalize data should appear like in the <font color=red>event_datafile_new.csv</font> after above code is executed.

<img src = 'Screenshot (177).png'>

#### Creating a cluster and connecting

In [6]:
# This makes a connection to a cassandra instance on the local host i.e. 
# 127.0.0.1

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    
    # To establish connection and begin executing queries, a session is setup.
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create a KeySpace

In [7]:
session.execute("""
CREATE KEYSPACE IF NOT EXISTS sparkify
WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")

#### Set this keyspace for creating tables.

In [8]:
session.set_keyspace('sparkify')
# session.use('sparkify')

### Queries to be analyze from the database are:
* Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
* Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
* Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## Query No. (1).

### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

To answer this question we will need to obtain (select) the artist name, song name, and song length from out table, and we will need to filter by sessionId and itemInSession. In CQL our query looks like:

SELECT artist, song_title, song_length FROM session_songs WHERE sessionId = 338 AND itemInSession = 4

- We will name our table session_songs
- Our primary key will consist of partition key sessionId, and clustering key itemInSession so that we can filter by this attributes later on.
- The columns of our table will be: sessionId, itemInSession, artist, song_title and song_length.

<font color='Blue'>Create table</font>

In [9]:
session.execute("""
    CREATE TABLE IF NOT EXISTS session_songs
    (sessionId int, itemInSession int, artist text, song_title text, song_length float,
    PRIMARY KEY(sessionId, itemInSession))
    """)

<font color='Blue'>Insert data into table</font>

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) #skip header
    
    for line in csvreader:
        query = """INSERT INTO session_songs(sessionId, ItemInSession, artist, 
        song_title, song_length)
        VALUES(%s, %s, %s, %s, %s)"""
        artist_name, user_name, gender, itemInSession, user_last_name, length, level, location, sessionId, song, userId = line
        
        session.execute(query, [int(sessionId), int(itemInSession), artist_name, song, float(length)])

#### Select to verify that the data have been inserted into each table

In [11]:
rows = session.execute("""
SELECT artist, song_title, song_length FROM session_songs
WHERE sessionId = 338 AND itemInSession = 4""")

df = pd.DataFrame(list(rows))

In [12]:
df.head()

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


## Query No. (2).

### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

To answer this question we will need to obtain (select) the artist name, song name, user name and user lastname from out table, we will need to filter by userId and sessionId, and order by itemInSession.

In CQL our query looks like:

SELECT itemInSession, artist, song, firstName, lastName FROM user_songs WHERE userId = 10 AND sessionId = 182

- We will name our table user_songs
- Our primary key will consist of partition key userId.
- Our clustering key will be sessionId and itemInSession so that our results are order by it.
- The columns of our table will be: userId, sessionId, itemInSession, artist, song and firstName and lastName.

<font color='Blue'>Create table</font>

In [13]:
session.execute("""
CREATE TABLE IF NOT EXISTS user_songs
(userId int, sessionId int, artist text, song text, firstName text, lastName text, itemInSession int,
PRIMARY KEY((userid),sessionId, itemInSession))""")

<font color='Blue'>Insert data into table</font>

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """INSERT INTO user_songs (userId, sessionId, artist, song, firstName, lastName, itemInSession)
        VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        
        session.execute(query, (int(userId), int(sessionId), artist, song, firstName, lastName, int(itemInSession)))

<font color='Blue'>Verify the results of query</font>

In [15]:
rows = session.execute("""SELECT itemInSession, artist, song, firstName, lastName
FROM user_songs WHERE userId = 10 AND sessionId = 182""")

df = pd.DataFrame(list(rows))

df

,iteminsession,artist,song,firstname,lastname
0,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,1,Three Drives,Greece 2000,Sylvie,Cruz
2,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


## Query No. (3).

### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

To answer this question we will need to obtain (select) the user first name and lastname from out table, and we will need to filter by song name. As user name and lastname, in large datasets, are not unique, we will add the column userId to uniquely identify users.

In CQL our query looks like:

SELECT firstName, lastName FROM app_history WHERE song = 'All Hands Against His Own'

- We will name our table app_history
- Our primary key will consist of partition key song, and clustering key userId. This uniquely identifies our rows.
- The columns of our table will be: song, firstName, lastName and userId.

<font color='Blue'>Create table</font>

In [16]:
session.execute("""CREATE TABLE IF NOT EXISTS app_history
(song text, firstName text, lastName text, userId int, PRIMARY KEY(song, userId))""")

<font color='Blue'>Insert data into table</font>

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) #skip header
    for line in csvreader:
        query = """INSERT INTO app_history (song, firstName, lastName, userId)
        VALUES (%s, %s, %s, %s)"""
        artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
        session.execute(query, (song, firstName, lastName, int(userId)))

<font color='Blue'>Verify the results of query</font>

In [18]:
rows = session.execute("""SELECT firstName, lastName FROM app_history
WHERE song = 'All Hands Against His Own'""")

df = pd.DataFrame(list(rows))

df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### We will drop the tables befoer closing out the sessions

In [19]:
session.execute("""DROP TABLE app_history""")

session.execute("""DROP TABLE user_songs""")

session.execute("""DROP TABLE session_songs""")

### Close the session and cluster connection

In [20]:
session.shutdown()
cluster.shutdown()